In [8]:
import src.generation.gamma as ga
import src.generation.multivariate_normal as mn

from src.salp.salp import SALP
from src.salp.salp import Node

from src.algorithms.round_robin import RoundRobin
from src.algorithms.best_fit import BestFit

import numpy as np
# from tqdm.notebook import tqdm
from tqdm import tqdm

In [9]:
from typing import List

def calculate_total_node_queueing_time(n: Node, power: float) -> float:
    stb = np.mean(n.ws)/power
    assert stb < 1, "Node {} is not stable".format(n.identity)
    coeff = stb/(1-stb)
    return coeff * np.sum(n.ws)
    
def calculate_all_nodes_queueing_time(nodes: List[Node], power: float) -> float:
    total_ct = 0
    for n in nodes:
        ct = calculate_total_node_queueing_time(n, power)
        total_ct += ct
    return total_ct

In [10]:
# TODO algorithms
# SALP
def salp_wrapper(N: int, load_vectors):
    return SALP(N, load_vectors, False)
# Best Fit
def bf_wrapper(N: int, load_vectors):
    return None
# Round robin
def rr_wrapper(N: int, load_vectors):
    return RoundRobin(N, load_vectors, False)

algorithms = [
    ('SALP', salp_wrapper),
#     ('BF', bf_wrapper),
    ('RR', rr_wrapper),
]

# Parametry dla iteracji
* F - ilość shardów
* N - ilość węzłów
* cor - poziom korelacji miedzy wektorami obciążenia
* ro - poziom obciążenia chmury
* size - długość wektorów obciążenia
* repeats - ilość powtórzeń pojedynczej symulacji
* parametry generatora do ustalenia

In [11]:
N = 100
F = 10*N
cor_range = np.arange(0.0, 1.01, 0.5)
load_range = [0.8, 0.9]
size=100
repeats=5
rng = (2,4)

In [12]:
def gamma_generator_factory(cor):
    return ga.Generator(F, size, cor, rng)

In [13]:
from typing import Tuple, List, Callable
from src.generation.base.generator import BaseGenerator

def evaluate_algorithms(N: int, node_power: float, algorithms: List[Tuple[str, Callable]], load_vectors: np.ndarray):
    for name, func in algorithms:
        cloud = func(N, load_vectors)
        try:
            total_ct = calculate_all_nodes_queueing_time(cloud.nodes, node_power)
        except AssertionError:
            total_ct = -1
        yield name, total_ct
        
def pipeline(N: int, size: int, repeats: int, cor_range: List[float], load_range: List[float], 
             generator_factory: Callable[[float], BaseGenerator], algotithms: List[Tuple[str, Callable]]):
    pbar = tqdm(total=repeats * len(cor_range) * len(load_range))
    for cor in cor_range:
        generator = generator_factory(cor)
        
        estimated_load = generator.get_estimated_cloud_load()
        estimated_node_load = estimated_load/size/N
#         print("Estimated node load", estimated_node_load)
        
        for ro in load_range:
            node_power = estimated_node_load/ro
            
            for _ in range(repeats):
                load_vectors = generator.generate_cloud_load_vectors()
#                 print(np.sum(load_vectors)/size/N)
                
                for res in evaluate_algorithms(N, node_power, algorithms, load_vectors):
                    yield((cor, ro) + res)
                pbar.update(1)
    pbar.close()
                
# for partial in pipeline(N, size, repeats, cor_range, load_range, gamma_generator_factory, algorithms):
#     print(partial)

In [14]:
import pandas as pd

gen = pipeline(N, size, repeats, cor_range, load_range, gamma_generator_factory, algorithms)
df = pd.DataFrame(gen, columns=['correlation', 'load', 'algorithm', 'value'])

df.head()

100%|██████████| 30/30 [00:47<00:00,  1.59s/it]


,correlation,load,algorithm,value
0,0.0,0.8,SALP,2.195028e+06
1,0.0,0.8,RR,2.370406e+06
2,0.0,0.8,SALP,2.229980e+06
3,0.0,0.8,RR,2.426357e+06
4,0.0,0.8,SALP,2.205539e+06
